In [17]:
import os
from torch import utils, Tensor, abs
import torch
import lightning as L
import pandas as pd
from lightning.pytorch.callbacks import EarlyStopping

from alpha_connect import AlphaZeroModelConnect4, MyDataset

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
data_path = os.path.join(os.getcwd(), "..", "data")
print(data_path)

/Users/alberttroussard/Documents/alpha-connect/notebooks/../data


In [20]:
# load data.csv


data = pd.read_csv(os.path.join(data_path, "../data/data2.csv"))
# shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

x = data.iloc[:, :-8].values

policy = data.iloc[:, -8:-1].values

value = data.iloc[:, -1].values


dataset = MyDataset(Tensor(x), Tensor(policy), Tensor(value))

loader = utils.data.DataLoader(dataset, batch_size=1, num_workers=9)

In [21]:
model = AlphaZeroModelConnect4()
model.load_state_dict(torch.load("../data/latest.pth"))

<All keys matched successfully>

In [22]:
# test the model
model.eval()

correct_value = 0
correct = 0
total = 0
moves = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
i = 0
display = 5
for x, policy_distribution, value in loader:
    i += 1
    policy = [i for i in range(7) if policy_distribution[0][i] != 0]
    predicted_policy, predicted_value = model(x)
    _, predicted = predicted_policy.max(1)
    moves[predicted.item()] += 1
    total += 1
    correct += 1 if (predicted in policy) else 0

    correct_value += 1 if torch.abs(predicted_value - value) < 0.5 else 0
    if display > 0:
        print(f"Policy: {list(policy_distribution)[0]}")
        print(f"Predicted policy: {predicted_policy[0]}")
        print(f"Value: {int(value)}")
        print(f"Predicted value: {float(predicted_value)}")
        display -= 1

    if i % 1000 == 0:
        print(f"Value Accuracy: {100*correct_value/total}")
        print(f"Accuracy: {100*correct/total}")
        print(moves)

print(f"Value Accuracy: {100*correct_value/total}")
print(f"Accuracy: {100*correct/total}")
print(moves)
model.train()

Policy: tensor([0., 1., 0., 0., 0., 0., 0.])
Predicted policy: tensor([ 3.5453e-04, -2.2529e-02,  2.0387e-02,  6.6434e-02,  8.6767e-01,
         9.2018e-03,  3.3982e-03], grad_fn=<SelectBackward0>)
Value: 1
Predicted value: -0.7925336360931396
Policy: tensor([0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333])
Predicted policy: tensor([ 0.0032, -0.0263,  0.0118,  0.2688,  0.6456,  0.1026, -0.0372],
       grad_fn=<SelectBackward0>)
Value: 1
Predicted value: 0.5067662596702576
Policy: tensor([0., 0., 0., 1., 0., 0., 0.])
Predicted policy: tensor([-0.0168, -0.0096,  0.0987,  0.7362,  0.0627,  0.1296, -0.0050],
       grad_fn=<SelectBackward0>)
Value: -1
Predicted value: -0.3587665855884552
Policy: tensor([0., 1., 0., 0., 0., 0., 0.])
Predicted policy: tensor([-0.0163,  0.0385, -0.0101,  0.4540,  0.2260,  0.2447, -0.0105],
       grad_fn=<SelectBackward0>)
Value: -1
Predicted value: 0.8817331194877625
Policy: tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.1667, 0.1667])
Predicted 

KeyboardInterrupt: 